In [1]:
import pandas as pd

data = pd.read_csv('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_gathered/all_human_annotations_processed.csv')


In [2]:
data.columns

Index(['review_point', 'paper_id', 'venue', 'focused_review', 'actionability',
       'actionability_label', 'actionability_label_type', 'batch',
       'grounding_specificity', 'grounding_specificity_label',
       'grounding_specificity_label_type', 'verifiability',
       'verifiability_label', 'verifiability_label_type', 'helpfulness',
       'helpfulness_label', 'helpfulness_label_type', 'professional_tone',
       'professional_tone_label', 'professional_tone_label_type',
       'valid_point', 'valid_point_label', 'valid_point_label_type'],
      dtype='object')

In [8]:
aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness']
samples_per_label = 0
data_type = 'silver'

# Initialize a dictionary to store sampled data for each aspect
def sample_data(data, aspects, samples_per_label, data_type='gold'):

    ## if sample per label is 0, return the data without sampling
    if samples_per_label == 0:
        ## put it to big number so we return all the data
        samples_per_label = 1_000_000
        
    sampled_data = {}
    for aspect in aspects:
        # Define the relevant columns
        label_column = f"{aspect}_label"
        label_type_column = f"{aspect}_label_type"
        # Filter rows where label type is 'gold'
        aspect_type_rows = data[data[label_type_column] == data_type]
        sampled_aspect = (
            aspect_type_rows
            .groupby(label_column, group_keys=False)
            .apply(lambda group: group.sample(n=samples_per_label, random_state=42) 
                if len(group) >= samples_per_label else group)
        )
        sampled_data[aspect] = sampled_aspect

    return sampled_data


sampled_data = sample_data(data, aspects,samples_per_label,data_type)

with pd.ExcelWriter(f"/home/abdelrahman.sadallah/mbzuai/review_rewrite/chatgpt/test_data/{data_type}_human_annotations.xlsx", mode='w', engine='xlsxwriter') as writer:
    for x in aspects:
        print(x, sampled_data[x].shape)
        ## save the data to excel  with ascpects as sheet nemes
        sampled_data[x].to_excel(writer, sheet_name=x, index=False)
        print(f"saved {x} data to excel")


/tmp/ipykernel_3372971/2990642416.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.sample(n=samples_per_label, random_state=42)
/tmp/ipykernel_3372971/2990642416.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.sample(n=samples_per_label, random_state=42)
/tmp/ipykernel_3372971/2990642416.py:23: DeprecationWarning: DataFrameGroupBy.apply operated

actionability (378, 23)
saved actionability data to excel
grounding_specificity (336, 23)
saved grounding_specificity data to excel
verifiability (381, 23)
saved verifiability data to excel
helpfulness (367, 23)
saved helpfulness data to excel
